In [ ]:
# Will iterate over all graphs on nn vertices from nn = start_n to nn = stop_n - 1.
# Can use shortcut_edges = edcount to start at edcount edges instead of starting with the complete graph on start_n vertices;
# this is useful if the kernel gets interrupted while working on graphs on 10 vertices since it takes multiple days to process
# all 11million+ graphs on 10 vertices.

# These parameters can be set here or can be set in a command line instance and then run the notebook from the command line.
# start_n = 2
# stop_n = 5
# shortcut_edges = False


# Do you want to remove previously calculated data?
remove_old_data = False

In [2]:
# Name of directory in which to save the data files.
path_prefix = 'data'

# Removes the results of previous calculations
if remove_old_data == True:
    import shutil
    
    try:
        shutil.rmtree(path_prefix)
    except FileNotFoundError:
        pass

In [3]:
# load the read/write functions
# Eventually replace this local load to loading from the public github repo
load('spectator_floor_number_read_write_functions.py')

In [4]:
%%cython # use cython to speed up computations -- This jupyter magic call cannot be in a .py script file

# use garbage collection to manage memory usage
import gc
gc.enable()


def usp_comp(amat):
    """ Return the spectator number of the graph, which is defined to be
    the size of the smallest unique-shortest-path complement of a graph.
    
    :param amat: A graph or adjacency matrix of a graph.
    """
    # Accept graph or matrix input
    try:
        # Get size from the number of columns, if amat is a matrix
        nn = amat.ncols()
    except AttributeError:
        # If amat is a graph or graph6_string, convert to adjacency matrix
        amat = amat.adjacency_matrix()
        # Get size from number of colummns
        nn = amat.ncols()
    if not amat:
        return nn - 1
    
    # Use fact that (i,j)-entry of A^k is the number of i-j walks of length k
    # to determine the length of the longest unique shortest path (the largest k
    # for which there exists an (i,j)-entry of A^k equal to 1.)
    A = amat + 2
    AA = A + 0
    compsize = nn - 1
    while min(min([yy for yy in xx if yy]) for xx in AA) == 1:
        compsize -= 1
        AA = AA*A
    
    return compsize



def Glabel(G):
    """
    Returns the graph6_string of the canonical labeling of graph G using the sage algorithm
    to determine the canonical labeling.
    
    :param G: A graph object.
    """
    return G.canonical_label(algorithm='sage').graph6_string()


def edgeclasses(G):
    """
    Generator function to generate the automorphism groups of the edges in graph G.
    
    :param G: A graph object.
    """
    Aut = G.automorphism_group()
    needs = {(xx[0], xx[1]): True for xx in G.edges()}
    while needs:
        anedge = next(iter(needs))
        yield anedge
        for xx in Aut.orbit(anedge, action='OnPairs'):
            if (xx[0], xx[1]) in needs:
                del needs[(xx[0], xx[1])]
            if (xx[1], xx[0]) in needs:
                del needs[(xx[1], xx[0])]

                
def deletions(G):
    """
    Generator function to generate the minors of G which are achievable by deleting a single edge
    from G.
    Returns the graph6_string of the canonical labeling.
    
    :param G: A graph object.
    """
    for ed in edgeclasses(G):
        H = G.copy()
        H.delete_edge(ed)
        yield Glabel(H)

        
def contractions(G):
    """
    Generator function to generate the minors of G which are achievable by either contracting a single
    edge in G or deleting a single isolated vertex in G.
    Returns the graph6_string of the canonical labeling.
    
    :param G: A graph object.
    """
    for ed in edgeclasses(G):
        H = G.copy()
        H.contract_edge(ed)
        yield Glabel(H)
    # Also covers isolated vertex deletion
    if 0 in G.degree():
        H = G.copy()
        H.delete_vertex(G.degree().index(0))
        yield Glabel(H)

In [5]:
def progressBar(iterable, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    import datetime
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    total = len(iterable)
    # Progress Bar Printing Function
    def printProgressBar(iteration):
        try:
            percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
            filledLength = int(length * iteration // total)
        except ZeroDivisionError:
            percent = 100
            filledLength = length
        
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} {prefix} |{bar}| {percent}% {suffix}', end = printEnd, flush=False)
    # Initial Call
    printProgressBar(0)
    # Update Progress Bar
    for i, item in enumerate(iterable):
        yield item
        printProgressBar(i + 1)
    # Print New Line on Complete
    print()

In [6]:
def spec_floor_first_pass():
    """
    Determines the minor floor of the spectator number for all graphs on nn vertices and
    edcount edges.
    """
    
    # Used for both the progress bar and controlling how often the partial_uspcm_dict and 
    # partial_seen_dict are written to files.
    one_percent = max(len(uspcm_dict[f'{nn}_verts'][f'{edcount}_edges']) // 100, 1)
    fraction_percent = max(len(uspcm_dict[f'{nn}_verts'][f'{edcount}_edges']) // 1000, 1)
    
    if fraction_percent < 500:
        save_percent = one_percent
    else:
        save_percent = fraction_percent
    
    # Used for progress bar
    num_graphs_worked = 0
    
    # Iterate over all graphs on nn vertices and edcount edges
    # amat is a graph6_string
    for amat in progressBar(uspcm_dict[f'{nn}_verts'][f'{edcount}_edges'], 
                            prefix = f"1st pass: nn={nn}, ee={edcount}:", 
                            suffix = '', length = 40):
        
        # Skip those graphs whose spectator minor floor has already been determined.
        if amat in seen_dict[f'{nn}_verts'][f'{edcount}_edges']:
            num_graphs_worked += 1
            continue
        
        # Skip the disconnected graphs since we can sum over connected components and save progress
        if Graph(amat).is_connected() == False:
            seen_dict[f'{nn}_verts'][f'{edcount}_edges'].add(amat)
            write_partial_uspcm_dict(nn, edcount, uspcm_dict, path_prefix)
            if edcount > 1:
                write_partial_uspcm_dict(nn, edcount-1, uspcm_dict, path_prefix)
            write_partial_seen_dict(nn, edcount, seen_dict, path_prefix)
            num_graphs_worked += 1
            continue
            
        # Current number that is potentially the spectator minor floor of graph amat
        mine = uspcm_dict[f'{nn}_verts'][f'{edcount}_edges'][amat]
        G = Graph(amat) # Generate the graph object whose graph6_string is amat

        # For each minor xx of G, compare the computer spectator number of xx to the newly
        # computed spectator number of G and update the claimed spectator minor floor of xx
        # if necessary
        for xx in deletions(G):
            # Note: This is actually updating the number for the minor xx
            if xx in uspcm_dict[f'{nn}_verts'][f'{edcount-1}_edges']:
                old = uspcm_dict[f'{nn}_verts'][f'{edcount-1}_edges'][xx]
                uspcm_dict[f'{nn}_verts'][f'{edcount-1}_edges'][xx] = min(mine, old)
            else:
                old = usp_comp(Graph(xx))
                uspcm_dict[f'{nn}_verts'][f'{edcount-1}_edges'][xx] = min(mine, old)
        for xx in contractions(G):
            # Note: This is actually updating the number for the minor xx
            xx_num_verts = Graph(xx).num_verts()
            xx_num_edges = Graph(xx).num_edges()
            old = uspcm_dict[f'{xx_num_verts}_verts'][f'{xx_num_edges}_edges'][xx]
            if old > mine:
                print(f'Exception found: {G.graph6_string()} has uspc {mine}', end='')
                print(f' with minor {Graph(xx).graph6_string()} of uspc {uspcm_list[-2][edplace][xx]}.')
                pass


        # Save the progress
        seen_dict[f'{nn}_verts'][f'{edcount}_edges'].add(amat)
        num_graphs_worked += 1
        # Save to file every one percent of the way through
        if num_graphs_worked % save_percent == 0:
            write_partial_uspcm_dict(nn, edcount, uspcm_dict, path_prefix)
            if edcount > 1:
                write_partial_uspcm_dict(nn, edcount-1, uspcm_dict, path_prefix)
            write_partial_seen_dict(nn, edcount, seen_dict, path_prefix)

In [6]:
# Functions to determine minimality in the second pass

def check_minimality(G, uspcm_dict):
    """
    Uses the dictionary of graphs and their spectator minor floor numbers calculated by the spec_floor_first_pass()
    function to determine which graphs are minor minimal with respect to the spectator minor floor number.
    """

    G, g6_str = get_canonical_graph(G)
    G_spec_floor = get_spectator_floor(G, uspcm_dict)

    for H in deletions(G):
        if get_spectator_floor(H, uspcm_dict) == G_spec_floor:
            # G is not minor minimal
            return None
    for H in contractions(G):
        if get_spectator_floor(H, uspcm_dict) == G_spec_floor:
            # G is not minor minimal
            return None
    else:
        # G is minor minimal
        return (g6_str, G_spec_floor)
    
    
    
def determine_minimals(nn, minimals_dict=None, uspcm_dict=None, save=False, path_prefix='data'):
    
    if uspcm_dict is None:
        uspcm_dict = get_full_uspcm_dict()
        
    if minimals_dict is None:
        minimals_dict_keys = [f'{kk}_spectators' for kk in range(10)]
        minimals_dict = dict(zip(minimals_dict_keys, [set() for kk in range(10)]))
        
    
    for edge_key in uspcm_dict.get(f'{nn}_verts'):
        if save==True:
            one_percent = max(len(uspcm_dict.get(f'{nn}_verts').get(edge_key)) // 100, 1)
            fraction_percent = max(len(uspcm_dict.get(f'{nn}_verts').get(edge_key)) // 1000, 1)

            if fraction_percent < 500:
                save_percent = one_percent
            else:
                save_percent = fraction_percent
            
            
        num_edges = edge_key.split('_')[0]
        print(f'Working on graphs on {nn} vertices and {num_edges} edges...')
        
        num_graphs_worked = 0
        
        for g6_str in progressBar(uspcm_dict.get(f'{nn}_verts').get(edge_key), 
                            prefix = f"2nd pass: nn={nn}, ee={num_edges}:", 
                            suffix = '', length = 40):
            if Graph(g6_str).is_connected():
#                 print(g6_str)
                result = check_minimality(g6_str, uspcm_dict)
                if result is not None:
                    G_spec_num = result[1]
                    minimals_dict.get(f'{G_spec_num}_spectators').add(g6_str)

            num_graphs_worked += 1
                
            if num_graphs_worked % save_percent == 0:
                with open(path_prefix +
                  f'/minimals_dict/minimals_dict_{nn}_verts_{num_edges}_edges.txt', 'w') as outfile:
                    outfile.write(str(minimals_dict))
                    
    return minimals_dict

In [7]:
import os

# Check to see if the data directory is set up; otherwise, create the data directory
if path_prefix not in os.listdir('.'):
    os.mkdir(path_prefix)
if 'minimals_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/minimals_dict')
if 'uspcm_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/uspcm_dict')
if 'full_uspcm_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/full_uspcm_dict')
if 'seen_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/seen_dict')
if 'completed_dict' not in os.listdir(path_prefix):
    os.mkdir(path_prefix + '/completed_dict')

    
# Initialize the uspcm_dict: The dictionary that starts with the spectator numbers and ends with containing
# the spectator minor floor numbers for the graphs.
# Initialize minimals_dict: The dictionary of graphs which are minimal with respect to the spectator minor floor number.
# Initialize seen_dict: The dictionary of graphs that have been processed by spec_floor_first_pass().
# Initialize completed_dict: The dictionary of graphs that have been processed by second_pass().
# All of these are nested dictionaries. uspcm_dict, seen_dict, and completed_dict have a first layer of keys of the form
# {nn}_verts and second layer of keys of the form {ee}_edges. For example, the set of graphs on 4 vertices and 5 edges 
# that have been processed by second_pass() is accessable via completed_dict['4_verts']['5_edges']
# The set of graphs which are spectator minor floor minimal for spectator number 3 is accessable via minimals_dict['3_spectators']
uspcm_dict, minimals_dict, seen_dict, completed_dict = get_spectator_number_dictionaries(path_prefix)

# The code depends on the partial_uspcm_dict for the graph on 0 vertices and for the graph on 1 vertex existing.
starter_uspcm_dict = {'0_verts':{}, '1_verts':{}}
starter_uspcm_dict['0_verts']['0_edges'] = {Glabel(Graph(0)): 0}
starter_uspcm_dict['1_verts']['0_edges'] = {Glabel(Graph(1)): 0}
write_partial_uspcm_dict(0, 0, starter_uspcm_dict, path_prefix)
write_partial_uspcm_dict(1, 0, starter_uspcm_dict, path_prefix)


# Iterate over all graphs on nn vertices from nn = start_n to nn = stop_n - 1.
for nn in range(start_n, stop_n):
    print(f'Working on graphs with {nn} vertices.')

    # Start with the complete graph on nn vertices, work down to the empty graph on nn vertices
    edcount = Integer((nn*(nn-1))/2)
    K_n = Graph(nn).complement()
    # Add to the uspcm_dict
    uspcm_dict[f'{nn}_verts'][f'{K_n.num_edges()}_edges'] = {Glabel(K_n): nn - 2}
    
    if shortcut_edges:   
        # Used if having to restart partway through graphs on 10 vertices since those 11million+ graphs take multiple days to process.
        edcount = shortcut_edges
    
    while edcount:
        # Working on graphs on nn vertices with decreasing number of edges down to 0 edges
        print(f'Processing graphs with {nn} vertices and {edcount} edges...')
        
        # First pass: set values, giving every superminor a chance.
        spec_floor_first_pass()
        
        # Save progress
        write_partial_uspcm_dict(nn, edcount, uspcm_dict, path_prefix)
        if edcount > 1:
            write_partial_uspcm_dict(nn, edcount-1, uspcm_dict, path_prefix)
#         write_full_uspcm_dict(uspcm_dict, path_prefix)
        write_partial_seen_dict(nn, edcount, seen_dict, path_prefix)
#         write_seen_dict(seen_dict, path_prefix)


#         # Second pass: check for minimality
# #         Something is incorrect with second_pass() resulting in not recognizing all of the minor minimal graphs
# #         Use the detecting_minimals.ipynb file instead
#         second_pass()
        
#         write_minimals_dict(nn, edcount, minimals_dict, path_prefix)
#         write_partial_completed_dict(nn, edcount, completed_dict, path_prefix)
#         write_completed_dict(completed_dict, path_prefix)

        edcount -= 1
print('Done.')

Working on graphs with 2 vertices.
Processing graphs with 2 vertices and 1 edges...
2021-09-16 17:46:04 1st pass: nn=2, ee=1: |████████████████████████████████████████| 100.0% 
2021-09-16 17:46:04 2nd pass: nn=2, ee=1: |████████████████████████████████████████| 100.0% 
Working on graphs with 3 vertices.
Processing graphs with 3 vertices and 3 edges...
2021-09-16 17:46:04 1st pass: nn=3, ee=3: |████████████████████████████████████████| 100.0% 
2021-09-16 17:46:04 2nd pass: nn=3, ee=3: |████████████████████████████████████████| 100.0% 
Processing graphs with 3 vertices and 2 edges...
2021-09-16 17:46:04 1st pass: nn=3, ee=2: |████████████████████████████████████████| 100.0% 
2021-09-16 17:46:04 2nd pass: nn=3, ee=2: |████████████████████████████████████████| 100.0% 
Processing graphs with 3 vertices and 1 edges...
2021-09-16 17:46:04 1st pass: nn=3, ee=1: |████████████████████████████████████████| 100.0% 
2021-09-16 17:46:04 2nd pass: nn=3, ee=1: |████████████████████████████████████████